# Testing ResNet

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load CIFAR100 dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])
cifar100_train = datasets.CIFAR100(root='./data_cifar100', train=True, download=True, transform=transform)
cifar100_test = datasets.CIFAR100(root='./data_cifar100', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# initialize data loader
BATCH_SIZE = 128
train_loader = DataLoader(cifar100_train, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(cifar100_test, batch_size = BATCH_SIZE, shuffle = False)

In [4]:
cifar100_test.data[0, :, :].shape

(32, 32, 3)

In [5]:
# set GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
# load ResNet50 model
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained = True)

Using cache found in C:\Users\fongc/.cache\torch\hub\pytorch_vision_v0.6.0
c:\Users\fongc\anaconda3\envs\ml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\fongc\anaconda3\envs\ml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# do transfer learning on model
model.fc = torch.nn.Linear(2048, 100)
model = model.to(device)

In [8]:
# freeze all layers except the last one
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [9]:
# define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr = 0.001, momentum = 0.9)

In [11]:
# train model
EPOCHS = 40
for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

tensor([[[[-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          ...,
          [-0.9281, -0.9281, -0.1365,  ...,  1.2562, -1.8957, -1.8957],
          [-0.2538, -0.4590,  0.0834,  ...,  1.3442, -1.8957, -1.8957],
          [ 0.0248, -0.0632, -0.3124,  ...,  1.3295, -1.8957, -1.8957]],

         [[-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
          [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
          [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
          ...,
          [-1.2706, -1.3929, -0.6744,  ...,  1.0533, -1.8975, -1.8975],
          [-0.6744, -0.8425, -0.3992,  ...,  1.1756, -1.8975, -1.8975],
          [-0.3839, -0.5062, -0.7355,  ...,  1.1450, -1.8975, -1.8975]],

         [[-1.5965, -1.5965, -1.5965,  ..., -1.5965, -1.5965, -1.5965],
          [-1.5965, -1.5965, -

KeyboardInterrupt: 

In [ ]:
# make predictions
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [ ]:
print("Accuracy:", correct / total)

Accuracy: 0.2759
